In [1]:
import pandas as pd
import nltk
import vaderSentiment
import re
import string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JZ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Read Data from CSV File:

In [2]:
path = r'C:\Users\JZ\Desktop\Data\JPMC_data_csv1.csv'

In [3]:
def load():
    pd.set_option("display.max_colwidth", None) #-1
    return pd.read_csv(path)

In [5]:
totalInfo = load().copy()

In [6]:
totalInfo

,Date,text,favorite_count,favorited,retweeted,lang,quoted_status,Stock_Name,sentiment,sentiment_class,compound score,Stock Price
0,10/24/2021,$BBIG $AMC $GME $DWAC $BENE âœ…ðŸ“ˆðŸ“ˆðŸ“ˆ ðŸ‘‰\n\n#DWAC #GME #BENE #BBIG #AMC https://t.co/PcMTemqch1,0,FALSE,FALSE,en,NaN,GME,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",neutral,0,169.8
1,10/24/2021,"RT @soFloTrading: SUNDAY GIVEAWAY TIME!\n\nOctober 18, 2021 - Weekly Trade Performance\n\nToday is the last day to lock in a FREE weekly trial!â€¦",0,FALSE,FALSE,en,NaN,GME,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'compound': 0.6825}",positive,0.68,169.8
2,10/24/2021,We are going through the beginning stages of everything Iâ€™ve said in the last year about the economy. Iâ€™m no geniusâ€¦ https://t.co/DaBpCTtQh6,0,FALSE,FALSE,en,NaN,GME,"{'neg': 0.095, 'neu': 0.905, 'pos': 0.0, 'compound': -0.296}",negative,-0.3,169.8
3,10/24/2021,$BBBY ðŸ‹ candidate! the base is minimum. ðŸš€\n\n$GME $AMC $BB $NOK $NAKD $CCIV $BTC $ETH $DOGE $TEL $SHIB $ADA $XTZâ€¦ https://t.co/c1RCHDpDkF,0,FALSE,FALSE,en,NaN,GME,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",neutral,0,169.8
4,10/24/2021,The largest mining company in the world\nGet Bonus 40% â¬‡ï¸\nhttps://t.co/kVp59h2td0\n\n$elon $doge $btc $tsla $amc $gmeâ€¦ https://t.co/I4ZfSukZaP,0,FALSE,FALSE,en,NaN,GME,"{'neg': 0.0, 'neu': 0.844, 'pos': 0.156, 'compound': 0.5423}",positive,NaN,169.8
...,...,...,...,...,...,...,...,...,...,...,...,...
26718,11/6/2021,@RobEducated $FB $MVRS Metaverse optionality isn’t really priced in at current ~22x P/E. I think I trust more Zuck… https://t.co/4w3U6cFthi,1,FALSE,FALSE,en,NaN,FB,"{'neg': 0.0, 'neu': 0.852, 'pos': 0.148, 'compound': 0.5106}",positive,0.51,341.13
26719,11/6/2021,"$FB $MRNA $NVAX $LAC $AAPL $NVDA\nMy calls , Weekly Options Gains :\n\n$FB 320p|Nov 05 | $1.55 to $3 &gt; 95% moved… https://t.co/c3k90lW7ye",1,FALSE,FALSE,en,NaN,FB,"{'neg': 0.0, 'neu': 0.906, 'pos': 0.094, 'compound': 0.34}",neutral,0.34,341.13
26720,11/6/2021,@801010athlete I am viewing Thursday move was due to metaverse announcements from $FB and $MSFT. The N in CANSLIM.… https://t.co/oemrZW4Sp3,2,FALSE,FALSE,en,NaN,FB,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",neutral,0,341.13
26721,11/6/2021,Good looking chart plays for the week. Enter long on PDH or a 5 minute squeeze. Nov. 12 Calls. \n$AAPL\n$AMZN\n$FB… https://t.co/bmNqvb2KsJ,1,FALSE,FALSE,en,NaN,FB,"{'neg': 0.0, 'neu': 0.811, 'pos': 0.189, 'compound': 0.5994}",positive,0.6,341.13


In [ ]:
#fill any NaN/invalid values
totalInfo = totalInfo.fillna(0)

In [ ]:
#get rid of any noisy data (compound score is zero)
filter_noise = (totalInfo['compound score '] != '0')
totalInfo = totalInfo.loc[filter_noise]

In [ ]:
totalInfo

Find if sentence has ... in it

In [ ]:
def remove_sentence(comment): #if text contains …, then it will be removed (… means truncated)
    print(comment.find('…'))
    if comment.find('…') > 0:
        #print("Remove")
        return 1
    else:
        #print("Don't remove")
        return -1
        

In [ ]:
totalInfo['remove'] = totalInfo['text'].map(lambda x: '1' if remove_sentence(x) == 1 else '-1')

Filter out sentences that are truncated:

In [ ]:
filter_truncated = totalInfo['remove'] == '-1'
full_comments_df = totalInfo.loc[filter_truncated]
full_comments_df

Before Filtering: 26722 rows

After Filtering: 15881 rows

60% of data remain

Modifed vaderSentiment Analyzer:

In [ ]:
def one_comment(sentences):
    sentence_list = []
    stocks_mentioned = {} 
    seen = False
    within_comment = nltk.sent_tokenize(sentences) #makes a list of all sentences in comment
    paragraph_sentiment = 0.0

    for sentence in within_comment:
        seen = False
        analyzer = SentimentIntensityAnalyzer() #Create analyzer object
        analyzer.polarity_scores(within_comment)
        vs = analyzer.polarity_scores(sentence) # sentiment of sentence
        words = nltk.word_tokenize(sentence) # makes a list of all the words within a sentence
        for word in words: #iterates through each word in sentence
            if word in all_stocks: #checks if stock name is a word
                if word not in stocks_mentioned: #adds word to dictionary
                    stocks_mentioned[word] = vs['compound'] #sets value of that stock sentiment to vs['compound']
                    seen = True  # Ex: "$AMC best stock $AMC" <--- if repetition in one sentence (we don't want to add to value)
                elif word in stocks_mentioned: #increases compound score
                    if seen == False: # Ex: "$AMC best stock $AMC" <--- repetition in one sentence
                        stocks_mentioned[word] += vs['compound'] 
        paragraph_sentiment += vs['compound']

    #print(within_comment, stocks_mentioned, paragraph_sentiment)
    return stocks_mentioned

In [ ]:
"""### Sentiment Analyzer"""

def company_scores(sentence): #returns a dictionary with company names in interest with compound score 
    sentiment_dict = one_comment(sentence)
    return sentiment_dict


def apply_vadersentiment(df):
    df['modified']=df['text'].apply(lambda x: company_scores(x))
    #df['sentiment_class']=df['sentiment'].apply(lambda x: 'positive' if x['compound']>0.5 else ('negative' if x['compound']<-0.5 else 'neutral'))
    #df['compound score']=df['sentiment'].apply(lambda x: x['compound'])
    return df

Obtain all stocks that we are interested in:

In [ ]:
stocks = totalInfo['Stock_Name'].to_numpy()
stocks

all_stocks = set(stocks)
all_stocks.remove('Stock_Name')

all_stocks

In [ ]:
apply_vadersentiment(full_comments_df)